In [122]:
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [123]:
!kaggle competitions download -c 2021-ml-midterm-p2

2021-ml-midterm-p2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [124]:
!unzip 2021-ml-midterm-p2.zip

Archive:  2021-ml-midterm-p2.zip
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [165]:
import pandas as pd
import numpy as np

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("sample_submit.csv")

In [166]:
x=train.drop(['index', 'Unnamed: 0','customerID','Churn'],axis=1)
test=test.drop(['index', 'Unnamed: 0','customerID'],axis=1)
y=train['Churn']

In [167]:
data_li=['gender', 
       'Partner', 'Dependents',  'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod']

In [168]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in data_li:
  le.fit(x[i])
  x[i]=le.transform(x[i])
  test[i]=le.transform(test[i])

In [169]:

print(x['TotalCharges'])

for i in range(len(x['TotalCharges'])):
  try:
    x['TotalCharges'][i]=float(x['TotalCharges'][i])
  except:
    
    print("%s is not number,it's index=%d"%(x['TotalCharges'][i],i))
    x['TotalCharges'][i]=0
x['TotalCharges']=x['TotalCharges'].astype(float)

print(x['TotalCharges'])

0       1734.65
1        3973.2
2       2869.85
3         238.5
4         119.5
         ...   
4783         95
4784     2198.3
4785     306.05
4786    1200.15
4787      457.3
Name: TotalCharges, Length: 4788, dtype: object
  is not number,it's index=280
  is not number,it's index=512


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


  is not number,it's index=2213
  is not number,it's index=2470
  is not number,it's index=2617
  is not number,it's index=3457
  is not number,it's index=4297
  is not number,it's index=4303
  is not number,it's index=4555
  is not number,it's index=4759
0       1734.65
1       3973.20
2       2869.85
3        238.50
4        119.50
         ...   
4783      95.00
4784    2198.30
4785     306.05
4786    1200.15
4787     457.30
Name: TotalCharges, Length: 4788, dtype: float64


In [170]:
import torch
seed=777
torch.manual_seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [171]:
for i in range(len(y)):
  if y[i]=='No':
    y[i]=0
  else:
    y[i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [172]:
y=y.astype("int")

In [173]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [174]:
x=torch.FloatTensor(np.array(x_std))
test=torch.FloatTensor(np.array(test_std))
y=torch.LongTensor(np.array(y))

In [196]:
linear1=torch.nn.Linear(19,32)
linear2=torch.nn.Linear(32,64)
linear3=torch.nn.Linear(64,32)
linear4=torch.nn.Linear(32,16)
linear5=torch.nn.Linear(16,2)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.3)

loss=torch.nn.CrossEntropyLoss()

In [197]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-0.4432,  0.5090,  0.6093,  0.0856,  0.0849, -0.2146,  0.1853,  0.4490,
         -0.5094,  0.1704, -0.1054,  0.4263, -0.2668, -0.0846,  0.2675,  0.0939],
        [ 0.0814,  0.1537, -0.4273, -0.5371,  0.0990, -0.1462,  0.3558,  0.1380,
          0.2621, -0.3046,  0.1807, -0.1464, -0.9306,  0.4191,  0.1172, -0.4769]],
       requires_grad=True)

In [198]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4,relu,dropout,linear5)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [199]:
model.train()

for stop in range(500):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.to(device))
  cost.backward()
  optimizer.step()
  if stop%100==0:
    print(stop,cost.item())

0 0.6186935305595398
100 0.46363624930381775
200 0.4422573149204254
300 0.42452800273895264
400 0.42026039958000183


In [200]:
pred=list()
with torch.no_grad():
  model.eval()
  y_test=model(test.to(device))
  a,b=torch.max(y_test,dim=1)
  pred.extend(b.tolist())

In [201]:
submit["Churn"]=pred

In [202]:
submit.to_csv("submit.csv",index=False)
!kaggle competitions submit -c 2021-ml-midterm-p2 -f submit.csv -m "message"

100% 7.99k/7.99k [00:00<00:00, 44.2kB/s]
Successfully submitted to 2021.ML.MidTerm.P2